In [1]:
import numpy as np
import pylab as plt
import os
import sys
sys.path.append('/mnt/d/shell/Wind Farm Evaluator/')
import Farm_Evaluator_Vec as ev
sys.path.pop()
import random, pylab as plt
import pandas as pd

In [2]:
turb_specs    =  {   
                         'Name': 'Anon Name',
                         'Vendor': 'Anon Vendor',
                         'Type': 'Anon Type',
                         'Dia (m)': 100,
                         'Rotor Area (m2)': 7853,
                         'Hub Height (m)': 100,
                         'Cut-in Wind Speed (m/s)': 3.5,
                         'Cut-out Wind Speed (m/s)': 25,
                         'Rated Wind Speed (m/s)': 15,
                         'Rated Power (MW)': 3
                  }
turb_diam      =  turb_specs['Dia (m)']
turb_rad       =  turb_diam/2 

In [3]:
# Turbine x,y coordinates
#turb_coords   =  ev.getTurbLoc(r'/content/drive/My Drive/shell/Shell_Hackathon Dataset/turbine_loc_test.csv')
   
# Load the power curve
power_curve   =  ev.loadPowerCurve('/mnt/d/shell/Shell_Hackathon Dataset/power_curve.csv')
    
# Pass wind data csv file location to function binWindResourceData.
# Retrieve probabilities of wind instance occurence.
wind_inst_freq =  ev.binWindResourceData(r'/mnt/d/shell/Shell_Hackathon Dataset/Wind Data/wind_data_2007.csv')    

In [4]:
# Doing preprocessing to avoid the same repeating calculations. Record 
# the required data for calculations. Do that once. Data are set up (shaped)
# to assist vectorization. Used later in function totalAEP.
n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t = ev.preProcessing(power_curve)

In [5]:
#entries = os.scandir('/mnt/d/shell/Shell_Hackathon Dataset/Wind Data/')

wind_inst = []
with os.scandir('/mnt/d/shell/Shell_Hackathon Dataset/Wind Data/') as entries:  
        for entry in entries:
            print(entry.name)
            wind_inst.append(ev.binWindResourceData(entry))

def avg_aep(L):
    
    turb_coords = np.array([[i*4000, j*4000] for i, j in L])
    
    return - ev.getAEP(turb_rad, turb_coords, power_curve, wind_inst[0], 
                                         n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t)
                                     

2017_2007.csv
combined.csv
wind_data_2007.csv
wind_data_2008.csv
wind_data_2009.csv
wind_data_2013.csv
wind_data_2014.csv
wind_data_2015.csv
wind_data_2017.csv


In [6]:
#generate a normalized grid of positions
import random

xpos = [x+50 for x in range(0, 3900, int(3900/9))]
xpos.pop()
xpos.append(3950)
#print(xpos)
norm_grid1 = []
for i in range(10):
    for j in range(10):
        norm_grid1.append([xpos[i]/4000, xpos[j]/4000])
random.shuffle(norm_grid1)

xpos = [x+50 for x in range(0, 3900, int(3900/8))]
xpos.pop()
xpos.append(3950)
#print(len(xpos))
norm_grid2 = []
for i in range(9):
    for j in range(9):
        norm_grid2.append([xpos[i]/4000, xpos[j]/4000])
random.shuffle(norm_grid2)

In [7]:
#generate an ensemble of n configs on the grid for initial configuration

ens_pos = []
for xx in range(250):
    L= random.sample(norm_grid1, 50)    
    ens_pos.append(L)

for xx in range(50):
    L= random.sample(norm_grid2, 50)    
    ens_pos.append(L)


In [8]:
#generate new grid with finer mesh 

xpos = [x+50 for x in range(0, 3900, int(3900/2000))]
xpos.pop()
xpos.append(3950)
print(len(xpos))
norm_grid = []
for i in range(3900):
    for j in range(3900):
        norm_grid.append([xpos[i]/4000, xpos[j]/4000])
random.shuffle(norm_grid)
# new = random.sample(norm_grid, 50)
# print(new)
# plt.scatter(*zip(*new)) 
# plt.show()
# print(avg_aep(new))

3900


In [9]:
# a = b[:] doesnt change b
def sortbyener(Li):
    ener = []
    for positions in Li:
        ener.append(avg_aep(positions))
    z = [x for _, x in sorted(zip(ener, Li), key=lambda pair: pair[0])]
    return z


def selection(Li, condi):   
    Li[:] = sortbyener(Li)
    if(condi):
        for i in range(250):
            Li.pop()
        while(len(Li) < 100):
            nam = random.uniform(0,1)
            if(nam < 0.5):
                Az = random.sample(norm_grid1, 50)
                Li.append(Az)
            else:
                Az = random.sample(norm_grid2, 50)
                Li.append(Az)
    else:
        num = 200
        for i in range(num):
            Li.pop()

def crossover(Li):
    while(len(Li) < 300):
      #change to 0 , len(Li -20)
        YY = [i for i in range(0, 100)]
        XX = random.sample(YY , 2)
        sum = list(Li[XX[0]] + Li[XX[1]]) 
        sum = list(set(map(tuple,sum)))      
        new = random.sample(sum, 50)
        condition = True
        for i in range(49):
            for j in range(i+1, 50):
                min_dist_sq = ((new[i][0] - new[j][0]) ** 2 + (new[i][1] - new[j][1]) ** 2)
                if (min_dist_sq < 0.01):
                    condition *= False
        if (condition):
            Li.append(new)
    # sort by energy
    Li = sortbyener(Li)

def mutation(Li):     
    num = random.randint(1, len(Li)-1)
    sel = Li.pop(num)
    num1 = random.randint(0, len(sel)-1)
    proo = random.uniform(0,1)
    if (proo <0.5):
        q = sel.pop(num1)
        new1 = random.choice(norm_grid)
        condition = True
        for i in range(len(sel)):       
            min_dist_sq = (new1[0] - sel[i][0])** 2 + (new1[1] - sel[i][1])** 2
            if (min_dist_sq < 0.01):
                condition *= False

        if(condition):
            sel.append(new1)
            Li.append(sel)
        else:
            sel.append(q)
            Li.append(sel)
    else:
        q = sel.pop(num1)
        new1 = [q[0] + random.choice(possib) + random.uniform(-delta, delta), q[1] + random.choice(possib) + random.uniform(-delta, delta)]
        box_cond = min(new1[0], new1[1]) > sigma and max(new1[0], new1[1]) < 1.0 - sigma
        condition = True
        for i in range(len(sel)):       
            min_dist_sq = (new1[0] - sel[i][0])** 2 + (new1[1] - sel[i][1])** 2
            if (min_dist_sq < 0.01):
                condition *= False

        if(condition and box_cond):
            sel.append(new1)
            Li.append(sel)
        else:
            sel.append(q)
            Li.append(sel)


In [10]:
totalSteps = 4000
overall_best_en = 0
overall_best = []
sigma = 0.0125
delta = 0.05
possib = [0.0, delta, -delta]
good = 10
overall_best_en = 0
for step in range(totalSteps):
    if (step%100 == 0):
        print(step, overall_best_en, overall_best)
    if (good - overall_best_en < 1):
        delta /= 10
        possib = [0.0, delta, -delta]
        selection(ens_pos, True)
    else:
        selection(ens_pos, False)
    crossover(ens_pos)
    for ixx in range(1000):
        if (ixx%400 == 0):
            ens_pos[:] = sortbyener(ens_pos)
        mutation(ens_pos)
    good = 0
    for positions in ens_pos:
        zz = avg_aep(positions)
        if (zz < overall_best_en):
            overall_best_en = zz
            overall_best[:] = positions 
        good += zz   
    good = good / len(ens_pos)
        
    print(step, delta, good, overall_best_en)


42797852
92 0.05 -547.6519373764039 -550.9683242797852
93 0.05 -547.6022930099484 -551.0727182006835
94 0.05 -547.8180980346684 -551.0727182006835
95 0.05 -547.9128220397946 -551.0727182006835
96 0.05 -547.6966940979004 -551.0727182006835
97 0.05 -547.9047733932497 -551.0727182006835
98 0.05 -547.8404009292606 -551.0727182006835
99 0.05 -547.8318141616827 -551.0727182006835
100 -551.0727182006835 [(0.0125, 0.0125), (0.256, 0.86475), (0.86475, 0.0125), (0.256, 0.4995), (0.13425, 0.256), (0.9875, 0.256), (0.256, 0.256), (0.0125, 0.13425), (0.743, 0.9875), (0.86475, 0.256), (0.4995, 0.9875), (0.9875, 0.13425), (0.9875, 0.0125), (0.0125, 0.37775), (0.256, 0.9875), (0.743, 0.0125), (0.37775, 0.9875), (0.0125, 0.743), (0.508, 0.72775), (0.84575, 0.385), (0.63325, 0.866), (0.42075, 0.797), (0.62125, 0.9875), (0.9875, 0.9875), (0.13425, 0.9875), (0.62125, 0.256), (0.8775, 0.62125), (0.37775, 0.0125), (0.9875, 0.4995), (0.0125, 0.9875), (0.9875, 0.62125), (0.743, 0.743), (0.0125, 0.256), (0.012

In [ ]:


print(overall_best_en)
print(overall_best)

plt.scatter(*zip(*overall_best)) 
plt.show()

xyz = np.array([[i*4000, j*4000] for i, j in overall_best])
df = pd.DataFrame(xyz)
df.to_csv('file9_17.csv', index=False, header=False)

In [ ]:

new11 = [[x*4000, y*4000] for x, y in overall_best]
ev.checkConstraints(new11, turb_diam)


In [ ]:
xyz = np.array([[i*4000, j*4000] for i, j in overall_best])
df = pd.DataFrame(xyz)
df.to_csv('file09_17.csv', index=False, header=False)

In [ ]:
#using combined
import matplotlib.pyplot as plt 


# L = [(0.121, 0.01275), (0.66225, 0.987), (0.121, 0.66225), (0.987, 0.987), (0.01275, 0.987), 
#      (0.3375, 0.44575), (0.7705, 0.987), (0.01275, 0.01275), (0.3375, 0.66225), (0.22925, 0.987), 
#      (0.22925, 0.01275), (0.3375, 0.987), (0.554, 0.01275), (0.01275, 0.44575), (0.01275, 0.3375),
#      (0.987, 0.7705), (0.66225, 0.66225), (0.44575, 0.3375), (0.44575, 0.01275), (0.987, 0.121), 
#      (0.987, 0.44575), (0.01275, 0.7705), (0.7705, 0.01275), (0.7705, 0.22925), (0.121, 0.987), 
#      (0.22925, 0.66225), (0.554, 0.987), (0.01275, 0.87875), (0.7705, 0.66225), (0.01275, 0.22925),
#      (0.01275, 0.121), (0.987, 0.3375), (0.987, 0.66225), (0.3375, 0.01275), (0.554, 0.3375),
#      (0.987, 0.01275), (0.66225, 0.01275), (0.987, 0.87875), (0.3375, 0.3375), (0.01275, 0.66225),
#      (0.01275, 0.554), (0.87875, 0.01275), (0.987, 0.554), (0.7705, 0.3375), (0.87875, 0.987),
#      (0.987, 0.22925), (0.44575, 0.66225), (0.87875, 0.66225), (0.554, 0.66225), (0.44575, 0.987)]
#   b = [[a[x][y] for y in range(len(a[0]))] for x in range(len(a))]

#535.7 one
# L = [(0.662, 0.9875), (0.33725, 0.0125), (0.0125, 0.4455), (0.4025, 0.5), (0.565, 0.4675),
#      (0.2725, 0.3375), (0.33725, 0.9875), (0.0125, 0.55375), (0.8575, 0.6625), (0.0125, 0.8785),
#      (0.77025, 0.0125), (0.9875, 0.55375), (0.12075, 0.0125), (0.0125, 0.662), (0.9875, 0.33725), 
#      (0.9875, 0.12075), (0.77025, 0.9875), (0.55375, 0.0125), (0.7275, 0.5), (0.0125, 0.77025),
#      (0.229, 0.9875), (0.4025, 0.7275), (0.8785, 0.0125), (0.3375, 0.1425), (0.9875, 0.229),
#      (0.825, 0.2725), (0.662, 0.0125), (0.0125, 0.12075), (0.9875, 0.9875), (0.55375, 0.9875), 
#      (0.9875, 0.0125), (0.229, 0.0125), (0.565, 0.305), (0.9875, 0.77025), (0.0125, 0.33725),
#      (0.9875, 0.4455), (0.0125, 0.0125), (0.0125, 0.9875), (0.24, 0.695), (0.565, 0.695),
#      (0.9875, 0.662), (0.4455, 0.0125), (0.2725, 0.5), (0.4455, 0.9875), (0.9875, 0.8785),
#      (0.435, 0.305), (0.8785, 0.9875), (0.12075, 0.9875), (0.7275, 0.6625), (0.0125, 0.229)]

L = [[overall_best[x][y] for y in range(len(overall_best[0]))] for x in range(len(overall_best))]

L = [list(x) for x in L]
old_L = []
best_fit = []
N = 50
sigma = 0.0125
best_energy = float('inf')
n_steps = 5000000
delta = 0.0125
#possib = [0.0, delta, -delta]
energy =  avg_aep(L)
best_energy = energy
best_fit = [[L[x][y] for y in range(len(L[0]))] for x in range(len(L))]
for step in range(n_steps):
    old_L = [[L[x][y] for y in range(len(L[0]))] for x in range(len(L))]
    a = random.choice(L)
    b = [a[0] + random.uniform(-delta, delta), a[1] + random.uniform(-delta, delta)]
    condition = True
    for i in range(len(L)):
        if (L[i][0] != a[0] and L[i][1] != a[1]):       
            min_dist_sq = (b[0] - L[i][0])** 2 + (b[1] - L[i][1])** 2
            if (min_dist_sq < 0.01):
                condition *= False 
    # make it 8 sigma
    box_cond1 = min(b[0], b[1]) > sigma and max(b[0], b[1]) < 1.0 - sigma
    if (box_cond1 and condition):
        #print('yay!')
        a[:] = b  
    else:
        L = [[old_L[x][y] for y in range(len(old_L[0]))] for x in range(len(old_L))]       

    new_energy = avg_aep(L)
    if new_energy < energy:        
        energy = new_energy
        if energy < best_energy:
           best_energy = energy
           best_fit = [[L[x][y] for y in range(len(L[0]))] for x in range(len(L))]

    if step % 1000 == 0:
        print (energy, step, best_energy)


print(best_fit)
print(best_energy)


In [ ]:
plt.scatter(*zip(*best_fit)) 
plt.show()

xyz = np.array([[i*4000, j*4000] for i, j in best_fit])
df = pd.DataFrame(xyz)
df.to_csv('file4.csv', index=False, header=False)